Sebelumnya kita telah belajar bagaimana melakukan tokenisasi dan membuat sequence dari setiap kalimat pada teks. Bayangkan jika Anda memiliki restoran dan Anda memiliki ratusan ribu review pelanggan. Anda menginginkan agar membedakan antara review positif dan negatif menjadi terotomasi sehingga Anda dapat memfilter review pelanggan dengan mudah. Pada submodul ini kita akan latihan untuk masalah ini. 

Untuk latihan kali ini kita akan menggunakan dataset **Yelp** yang berisi review dari beberapa restoran di Amerika Serikat. Dataset terdiri dari 2 kelas yaitu 0 dan 1 yang menunjukkan apakah review tersebut positif atau negatif. Dataset dapat diunduh pada [tautan](https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-setps://) ini yah. 

Setelah dataset diunduh, kita load dataset pada Colaboratory. Dan pastinya kita buat dataframe dari dataset kita agar lebih mudah diproses.

In [4]:
import pandas as pd

df = pd.read_csv('/content/yelp_labelled.txt', names=['sentences', 'label'], sep='\t')

Untuk melihat 5 sampel terakhir dari dataset kita, panggil fungsi `tail()`. 

In [5]:
df.tail()

,sentences,label
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


Setelah itu kita bagi dataset menjadi 'train' dan 'test' set.

In [ ]:
from sklearn.model_selection import train_test_split

kalimat = df['sentences'].values
y = df['label'].values
kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, test_size=0.2)

Agar teks dapat dipahami oleh model, kita harus lakukan tokenisasi. Gunakan fungsi `tokenizer` pada data latih dan data test. Jangan lupa gunakan fungsi `pad_sequences` agar setiap sequence sama panjang.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=250, oov_token='x')
tokenizer.fit_on_texts(kalimat_latih)
tokenizer.fit_on_texts(kalimat_test)

sekuens_latih = tokenizer.texts_to_sequences(kalimat_latih)
sekuens_test = tokenizer.texts_to_sequences(kalimat_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

Untuk arsitektur yang akan digunakan adalah *layer embedding*, dengan argumen pertama sesuai dengan jumlah vocabulary/kata yang kita pakai pada tokenizer. Argumen selanjutnya adalah *dimensi embedding*, dan *input_length* yang merupakan panjang dari sequence. Nah di kita tidak menggunakan layer *Flatten* melainkan kita menggantinya dengan **GlobalAveragePooling1D**. Fungsi ini bekerja lebih baik pada kasus NLP dibanding *Flatten*.

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(250, 16, input_length=20),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Setelah arsitektur model dibentuk, dan loss function serta optimizer ditentukan, kita dapat memulai pelatihan model kita. Di sini penulis menggunakan 30 epoch. Anda bebas bereksperimen dengan nilai yang lain.

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, y_latih, epochs=num_epochs,
                    validation_data=(padded_test, y_test),
                    verbose=2)

# Hasil dari pelatihan model kita menunjukkan akurasi yang cukup baik dengan data yang sangat sedikit.

Epoch 1/30
25/25 - 0s - loss: 0.1975 - accuracy: 0.9262 - val_loss: 0.8680 - val_accuracy: 0.7000
Epoch 2/30
25/25 - 0s - loss: 0.1932 - accuracy: 0.9262 - val_loss: 0.8343 - val_accuracy: 0.6900
Epoch 3/30
25/25 - 0s - loss: 0.1901 - accuracy: 0.9275 - val_loss: 0.8379 - val_accuracy: 0.6950
Epoch 4/30
25/25 - 0s - loss: 0.1912 - accuracy: 0.9250 - val_loss: 0.8583 - val_accuracy: 0.6950
Epoch 5/30
25/25 - 0s - loss: 0.1930 - accuracy: 0.9262 - val_loss: 0.8530 - val_accuracy: 0.6850
Epoch 6/30
25/25 - 0s - loss: 0.1903 - accuracy: 0.9275 - val_loss: 0.8672 - val_accuracy: 0.7000
Epoch 7/30
25/25 - 0s - loss: 0.1877 - accuracy: 0.9337 - val_loss: 0.8692 - val_accuracy: 0.6850
Epoch 8/30
25/25 - 0s - loss: 0.1871 - accuracy: 0.9325 - val_loss: 0.8786 - val_accuracy: 0.6900
Epoch 9/30
25/25 - 0s - loss: 0.1877 - accuracy: 0.9287 - val_loss: 0.9155 - val_accuracy: 0.6950
Epoch 10/30
25/25 - 0s - loss: 0.1858 - accuracy: 0.9300 - val_loss: 0.8867 - val_accuracy: 0.6800
Epoch 11/30
25/25 -

# Bagaimana? mudah bukan membuat model untuk klasifikasi teks. DI submodul selanjutnya kita akan belajar menggunakan layer LSTM dalam kasus NLP.